In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
reference = """Two people, Jenny and Peter, are having a conversation. Jenny lives in Dubai with her wife and works in a factory, but would rather work in an office. Peter works in a supermarket as a manager and enjoys his job. Jenny has a lot of friends, including around thirty best friends, while Peter has only one best friend. Jenny's ear hurts every time she takes a plane, and doctors couldn't fix the problem. The conversation ends with the two saying goodbye."""

# Pegasus Model

In [ ]:
PegSummary = """The following is a transcript of the conversation between Jenny and Peter, which took place in Dubai last week. How long do you live there? I have been married for three years). Does your wife know what's going on at home or does she go out to dinner with her husband every night?! Every time that plane goes up... my ear keeps getting bigger And then it gets even worse!"""

In [ ]:
rouge.get_scores(PegSummary, reference)

[{'rouge-1': {'r': 0.2833333333333333,
   'p': 0.24285714285714285,
   'f': 0.26153845656804736},
  'rouge-2': {'r': 0.05128205128205128,
   'p': 0.057971014492753624,
   'f': 0.05442176372622565},
  'rouge-l': {'r': 0.21666666666666667,
   'p': 0.18571428571428572,
   'f': 0.19999999502958593}}]

# Bart Model

In [ ]:
BartSummary = """Jenny lives in Abu Dhabi and works in a supermarket. Peter lives in Dubai and works as a factory manager. He has been married to his wife for three years. Peter says he has lost a lot of his best friends. He says he feels sorry for Jenny, who must be lonely. Jenny says she will share her friends with you, but she has to go. She says her ear is killing her every time a plane goes up."""

In [ ]:
rouge.get_scores(BartSummary, reference)

[{'rouge-1': {'r': 0.4, 'p': 0.43636363636363634, 'f': 0.4173912993572779},
  'rouge-2': {'r': 0.14102564102564102,
   'p': 0.14666666666666667,
   'f': 0.14379084467512512},
  'rouge-l': {'r': 0.4, 'p': 0.43636363636363634, 'f': 0.4173912993572779}}]

# T5 Model

In [ ]:
T5Summary = """The BBC's weekly The Boss series profiles different people from around the world. This is a full transcript of the story of Jenny, who has been married for seven years in Dubai. Here, she explains how she and her husband have been talking to us about their experiences and why they're lonely."""

In [ ]:
rouge.get_scores(T5Summary, reference)

[{'rouge-1': {'r': 0.18333333333333332,
   'p': 0.2391304347826087,
   'f': 0.20754716489854053},
  'rouge-2': {'r': 0.01282051282051282,
   'p': 0.0196078431372549,
   'f': 0.01550387118803104},
  'rouge-l': {'r': 0.16666666666666666,
   'p': 0.21739130434782608,
   'f': 0.18867924037023862}}]

# Bert Embeddings for checking Cosine Similarity

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentences = [PegSummary, BartSummary, T5Summary]

In [ ]:
sentence_embeddings = sbert_model.encode(sentences)

In [ ]:
ref_vec = sbert_model.encode([reference])[0]

In [ ]:
from scipy.spatial import distance
for sent in sentences:
  sim = distance.cosine(ref_vec, sbert_model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  The following is a transcript of the conversation between Jenny and Peter, which took place in Dubai last week. How long do you live there? I have been married for three years). Does your wife know what's going on at home or does she go out to dinner with her husband every night?! Every time that plane goes up... my ear keeps getting bigger And then it gets even worse! ; similarity =  0.2916005253791809
Sentence =  Jenny lives in Abu Dhabi and works in a supermarket. Peter lives in Dubai and works as a factory manager. He has been married to his wife for three years. Peter says he has lost a lot of his best friends. He says he feels sorry for Jenny, who must be lonely. Jenny says she will share her friends with you, but she has to go. She says her ear is killing her every time a plane goes up. ; similarity =  0.15220457315444946
Sentence =  The BBC's weekly The Boss series profiles different people from around the world. This is a full transcript of the story of Jenny, who 

# InferSent Embeddings for checking Cosine Similarity

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

In [ ]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/content/GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [ ]:
model.build_vocab(sentences, tokenize=True)

In [ ]:
ref_vec = model.encode([reference])[0]

In [ ]:
similarity = []
for sent in sentences:
  sim = distance.cosine(ref_vec, model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)


Sentence =  The following is a transcript of the conversation between Jenny and Peter, which took place in Dubai last week. How long do you live there? I have been married for three years). Does your wife know what's going on at home or does she go out to dinner with her husband every night?! Every time that plane goes up... my ear keeps getting bigger And then it gets even worse! ; similarity =  0.3921583890914917
Sentence =  Jenny lives in Abu Dhabi and works in a supermarket. Peter lives in Dubai and works as a factory manager. He has been married to his wife for three years. Peter says he has lost a lot of his best friends. He says he feels sorry for Jenny, who must be lonely. Jenny says she will share her friends with you, but she has to go. She says her ear is killing her every time a plane goes up. ; similarity =  0.02439248561859131
Sentence =  The BBC's weekly The Boss series profiles different people from around the world. This is a full transcript of the story of Jenny, who 

# Spacy Doc Similarity

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
reference1 = nlp(u"Two people, Jenny and Peter, are having a conversation. Jenny lives in Dubai with her wife and works in a factory, but would rather work in an office. Peter works in a supermarket as a manager and enjoys his job. Jenny has a lot of friends, including around thirty best friends, while Peter has only one best friend. Jenny's ear hurts every time she takes a plane, and doctors couldn't fix the problem. The conversation ends with the two saying goodbye.")
PegSummary1 = nlp(u"The following is a transcript of the conversation between Jenny and Peter, which took place in Dubai last week. How long do you live there? I have been married for three years). Does your wife know what's going on at home or does she go out to dinner with her husband every night?! Every time that plane goes up... my ear keeps getting bigger And then it gets even worse!")
BartSummary1 = nlp(u"Jenny lives in Abu Dhabi and works in a supermarket. Peter lives in Dubai and works as a factory manager. He has been married to his wife for three years. Peter says he has lost a lot of his best friends. He says he feels sorry for Jenny, who must be lonely. Jenny says she will share her friends with you, but she has to go. She says her ear is killing her every time a plane goes up.")
T5Summary1 = nlp(u"The BBC's weekly The Boss series profiles different people from around the world. This is a full transcript of the story of Jenny, who has been married for seven years in Dubai. Here, she explains how she and her husband have been talking to us about their experiences and why they're lonely.")

In [ ]:
print(reference1.similarity(PegSummary1))
print(reference1.similarity(BartSummary1))
print(reference1.similarity(T5Summary1))

0.8468214366136863
0.9250934543737741
0.934566242904275
